In [1]:
"""
Notes for reading commments in this file:
1) if there's a variable called 'count', I will used $count to denote the value stored in the variable count
2) 'pitch' refers to a specific type of drum sound. for example, pitch 36 may refer to a snare sound (or whatever percussion sound it actually is)
"""

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os

"""
Due to the nature of file systems in Google Drive, the 'My Drive' directory is actually
what you see when you open google drive. And to access it through colab (for read and write),
you need to run the mount command (which is in the first code segment at the top of this ipynb file).

curdir has the directory that I put this ipynb file in.
savedir is the directory I'm saving all the .mid and .npy files to.

Notice that the last line of one_hot_to_midi() function uses savedir to save the midi file.
Saving the .npy files also uses savedir
"""
curdir = os.getcwd()+'/gdrive/My Drive/Colab Notebooks/'
savedir = os.getcwd()+'/gdrive/My Drive/Colab Notebooks/MIDI/'

In [48]:
"""
Just run this code to download all the 2-bar midi tracks.
"""

import tensorflow_datasets as tfds
import tensorflow as tf

import numpy as np

# Colab/Notebook specific stuff
import IPython.display
from IPython.display import Audio
from google.colab import files

# Magenta specific stuff
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
from magenta import music as mm
from magenta.music import midi_synth
from magenta.music.sequences_lib import concatenate_sequences
from magenta.models.music_vae import data
from magenta.protobuf import music_pb2

# Some midi files come by default from different instrument channels
# Quick and dirty way to set midi files to be recognized as drums
def set_to_drums(ns):
  for n in ns.notes:
    n.instrument=9
    n.is_drum = True

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

# Calculate quantization steps but do not remove microtiming
def quantize(s, steps_per_quarter=4):
  return mm.sequences_lib.quantize_note_sequence(s,steps_per_quarter)

def is_4_4(s):
  ts = s.time_signatures[0]
  return (ts.numerator == 4 and ts.denominator ==4)

print("Download MIDI data...")

dataset_2bar = tfds.as_numpy(tfds.load(
    name="groove/2bar-midionly",
    split=tfds.Split.TRAIN,
    try_gcs=True))

trax = [features for features in dataset_2bar]
styles = [t['style'] for t in trax]
primary_styles = [s['primary'] for s in styles]
unique, counts = np.unique(primary_styles, return_counts=True)
print(unique[np.argmax(counts)]) # 16 = rock
rock_idx = []
for i in range(len(primary_styles)):
  if primary_styles[i] == 16:
    rock_idx.append(i)

print("Download MIDI data again...")

dataset_2bar = tfds.as_numpy(tfds.load(
    name="groove/2bar-midionly",
    split=tfds.Split.TRAIN,
    try_gcs=True))

dev_sequences = [quantize(mm.midi_to_note_sequence(features["midi"])) for features in dataset_2bar]
print("Filtering out rock-style tracks...")
dev_sequences = [dev_sequences[i] for i in rock_idx]

_ = [set_to_drums(s) for s in dev_sequences]
dev_sequences = [s for s in dev_sequences if is_4_4(s) and len(s.notes) > 0 and s.notes[-1].quantized_end_step > mm.steps_per_bar_in_quantized_sequence(s)]

Download MIDI data...
16
Download MIDI data again...


In [0]:
"""
maps all possible pitches down to 9 different pitches
"""
pitch_map = {
    36 : 36,
    38 : 38,
    40 : 38,
    37 : 38,
    48 : 50,
    50 : 50,
    45 : 47,
    47 : 47,
    43 : 43,
    58 : 43,
    46 : 46,
    26 : 46,
    42 : 42,
    22 : 42,
    44 : 42,
    49 : 49,
    55 : 49,
    57 : 49,
    52 : 49,
    51 : 51,
    59 : 51,
    53 : 51
}

"""
maps pitches to their respective indices in the one_hot encoding of a track
"""
adjusted_pitch_to_index_map = {
    36 : 0,
    38 : 1,
    50 : 2,
    47 : 3,
    43 : 4,
    46 : 5,
    42 : 6,
    49 : 7,
    51 : 8,
    0 : 9 # this isn't used but I put it here just to be safe
}

"""
maps indices in the one_hot matrix to their respective pitches
"""
index_to_pitch_map = {
    0 : 36,
    1 : 38,
    2 : 50,
    3 : 47,
    4 : 43,
    5 : 46,
    6 : 42,
    7 : 49,
    8 : 51,
    9 : 0 # value 0 is not defined on channel 9 so it's just silence (channel 9 is for drums)
}

def convert_pitch(pitch):
    return pitch_map[pitch]

"""
Input: tempo in terms of microseconds per quarter/fourth beat
Output: tempo in terns of beats per minute (bpm)
"""
def convert_tempo(tempo):
    return 60 / tempo / 1000000

def pitch_to_index(pitch):
    return adjusted_pitch_to_index_map[pitch]

def index_to_pitch(index):
    return index_to_pitch_map[index]

"""
Input: time in seconds, tempo
output: n, for the nth sixteenth beat that the time value falls into (for that specific tempo) n is 0...15
"""
def time_to_sixteeth_interval(time, tempo):
  return np.floor(time / (60 / (tempo * 4)))

def filter_rock_tracks(ds):
  return ds[rock_idx]

In [47]:
np.max(rock_idx)
len(primary_styles)
len(trax)

18163

In [0]:
"""
num_tracks refers to the number of 2-bar midi tracks in the loaded dataset
total_beats is the number of 16th beats in each track, which should be 32 in our case (2 bars of 16 16th beats each)
num_instruments is 9 + silence = 10
"""
num_tracks = len(dev_sequences)
num_instruments = 10
total_beats = 32

"""
Limits the number of tracks to encode, just for testing purposes. If you want to encode all the tracks,
just remove it from the line that reads:
for i, midi_track in enumerate(dev_sequences[:limiter]):

Change it to:
for i, midi_track in enumerate(dev_sequences):

In the training set, there are 16389 tracks.
"""
limiter = 16389

"""
tracks_one_hot stores a list of one-hot encodings for each track in a 3-d structure.
Each one-hot encoding has 32x10 structure - self-explanatory.

tracks_seq stores a list of pitch sequences in a 2-d structure.
For example, tracks_seq[0] may contain:
[36,0,42,0,36,0,42,36,36,0,38,...] (32 elements total)

and tracks_seq would contain $num_tracks of these types of sequences
"""
tracks_one_hot = np.zeros((limiter, num_instruments, total_beats))
tracks_seq = np.zeros((limiter, total_beats))

"""
The following code will get one_hot encodings for $limiter 2-bar midi tracks.
After running it, the first $limiter entries in tracks_one_hot and tracks_seq
will have values. The rest of the entries will be zeroes.
"""

for i, midi_track in enumerate(dev_sequences[:limiter]): # remove limiter, if you want
  note_intervals = [[] for _ in range(total_beats)] # list of possible (note, velocity) pairs for each sixteenth beat
  tempo = midi_track.tempos
  
  if len(tempo) == 1:
    tempo = tempo[0]
  else: # if there are 0, or >2 tempo markings, skip the track for now
    continue

  # Puts all the notes into corresponding 16th beat interval groups
  for note in midi_track.notes:
    note_pitch = convert_pitch(note.pitch)
    note_velocity = note.velocity

    note_start_step = note.quantized_start_step
    note_end_step = note.quantized_end_step

    # note_intervals[sixteenth beat index] appends a (pitch, velocity) tuple
    if note_start_step > 31:
      note_start_step = 31
    note_intervals[note_start_step].append((note_pitch, note_velocity))

  # Choose 1 note in each of the 16th beat interval groups, based on highest velocity
  for b in range(total_beats):
    candidates = note_intervals[b]
    
    converted_note_index = 9 # 9 is default, for "no drum"
    if len(candidates) > 0:
      highest_velocity_index = np.argmax([c[1] for c in candidates])
      selected_note = candidates[highest_velocity_index]
      converted_note_index = pitch_to_index(selected_note[0])
    tracks_one_hot[i, converted_note_index, b] = 1
    tracks_seq[i, b] = index_to_pitch(converted_note_index)

In [51]:
tracks_seq[0]

array([ 0.,  0., 42.,  0., 38.,  0., 42., 38., 46., 38.,  0.,  0., 38.,
       38., 38., 38., 49.,  0., 46.,  0., 38.,  0., 49.,  0., 42.,  0.,
       38.,  0., 42.,  0., 36.,  0.])

In [0]:
"""
Saves tracks_one_hot to tracks_one_hot.npy
Saves tracks_seq to tracks_seq.npy
"""

np.save(savedir + 'tracks_one_hot_' + str(limiter) + '.npy', tracks_one_hot)
np.save(savedir + 'tracks_seq_' + str(limiter) + '.npy', tracks_seq)

In [0]:
from mido import Message, MidiFile, MidiTrack

"""
Input: one hot track, aka a single element from tracks_one_hot.
       filename, ending with '.mid'
Purpose: Saves a midi file
Output: None
Example Usage: one_hot_to_midi(tracks_one_hot[i], midi_filename='song.mid')
"""
def one_hot_to_midi(one_hot, midi_filename = 'song.mid'):
  mid = MidiFile()
  track = MidiTrack()
  mid.tracks.append(track)

  pitch = 36
  duration = 128
  velocity = 64

  track.append(Message('program_change', program=12, time=0))

  #for sb in one_hot:
  for i in range(one_hot.shape[1]):
    sb = one_hot[:,i]
  
    pitch = int(index_to_pitch(np.argmax(sb)))
    track.append(Message('note_on', channel=9, note=pitch, velocity=64, time=duration))
    track.append(Message('note_off', channel=9, note=pitch, velocity=127, time=duration))

  mid.save(savedir + midi_filename)

In [0]:
"""
Example: Saves $limiter midi files to the curdir directory
"""
for i in range(limiter):
  one_hot_to_midi(tracks_one_hot[i], midi_filename='song'+str(i)+'.mid')